In [1]:
import keras

Using TensorFlow backend.


Neural style transfer consists in applying the "style" of a reference image to a target image, while conserving the "content" of the target image

We define a loss function to specify what we want to achieve: conserve the "content" of the original image, while adopting the "style" of the reference image. If we were able to mathematically define content and style, then an appropriate loss function to minimize would be the following:

loss = distance(style(reference_image) - style(generated_image)) +
       distance(content(original_image) - content(generated_image))

- distance is a norm function such as the L2 norm
- content is a function that takes an image and computes a representation of its "content"
- style is a function that takes an image and computes a representation of its "style".

Minimizing this loss would cause style(generated_image) to be close to style(reference_image), while content(generated_image) would be close to content(generated_image), thus achieving style transfer as we defined it.

Using a pre-trained convnet to define a loss that will:

- Preserve content by maintaining similar high-level layer activations between the target content image and the generated image. The convnet should "see" both the target image and the generated image as "containing the same things".

- Preserve style by maintaining similar correlations within activations for both low-level layers and high-level layers. Indeed, feature correlations capture textures: the generated and the style reference image should share the same textures at different spatial scales.

In [3]:
from keras.preprocessing.image import load_img, img_to_array

target_image_path = 'Pictures/dog.jpg'
style_reference_image_path = 'Pictures/abstract.jpg'

width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

### VGG19

VGG19 model, with weights pre-trained on ImageNet
Process:
- Set up a network that will compute VGG19 layer activations for the style reference image, the target image, and the generated image at the same time.
- Use the layer activations computed over these three images to define the loss function, which we will minimize in order to achieve style transfer.
- Set up a gradient descent process to minimize this loss function.

In [4]:
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height,img_width))
    img = img_to_array(img) #Converts a PIL Image instance to a Numpy array.
    img = np.expand_dims(img, axis=0) #Expand the shape of an array.
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

### VGG19 Network

It takes as input a batch of three images: the style reference image, the target image, and a placeholder that will contain the generated image.
The style reference and target image are static, and thus defined using K.constant, while the values contained in the placeholder of the generated image will change over time.

In [7]:
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))

combination_image = K.placeholder((1,img_height,img_width,3))

input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet',include_top=False)

Instructions for updating:
Colocations handled automatically by placer.
80142336/80134624 [==============================] - 131s 2us/step


In [8]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.3-neural-style-transfer.ipynb